<a href="https://colab.research.google.com/github/ckjen168/LLMColab/blob/main/stk_ch06_otc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-06 個股分析機器人

### 1️⃣ 安裝及匯入套件

In [ ]:
!pip install openai
!pip install yfinance==0.2.38
from openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import numpy as np
import datetime as dt # 時間套件
import requests
from bs4 import BeautifulSoup

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


### 2️⃣ 輸入 OpenAI API KEY

In [ ]:
import getpass # 保密輸入套件
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key = api_key) # 建立 OpenAI 物件

請輸入金鑰：··········


### 3️⃣ 取得股價資料

In [ ]:
# 從 yfinance 取得一周股價資料
def stock_price(stock_id="大盤", days = 10):
  if stock_id == "大盤":
    stock_id="^TWII"
  else:
    stock_id

  end = dt.date.today() # 資料結束時間
  start = end - dt.timedelta(days=days) # 資料開始時間
  # 下載資料
  df = yf.download(stock_id, start=start)

  # 更換列名
  df.columns = ['開盤價', '最高價', '最低價',
                '收盤價', '調整後收盤價', '成交量']

  data = {
    '日期': df.index.strftime('%Y-%m-%d').tolist(),
    '收盤價': df['收盤價'].tolist(),
    '每日報酬': df['收盤價'].pct_change().tolist(),
    '漲跌價差': df['調整後收盤價'].diff().tolist()
    }

  return data

print(stock_price("4205.TWO"))

[*********************100%%**********************]  1 of 1 completed

{'日期': ['2024-02-23', '2024-02-26', '2024-02-27', '2024-02-29', '2024-03-01', '2024-03-04'], '收盤價': [49.650001525878906, 49.650001525878906, 49.849998474121094, 49.79999923706055, 49.75, 50.0], '每日報酬': [nan, 0.0, 0.004028135792462084, -0.0010029937530791067, -0.0010040007595690792, 0.005025125628140614], '漲跌價差': [nan, 0.0, 0.1999969482421875, -0.049999237060546875, -0.049999237060546875, 0.25]}


### 4️⃣ 取得基本面資料

In [ ]:
# 基本面資料
def stock_fundamental(stock_id= "大盤"):
  if stock_id == "大盤":
      return None

  stock = yf.Ticker(stock_id)

  # 營收成長率
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)

  # 每季EPS
  quarterly_eps = np.round(stock.quarterly_financials.loc["Basic EPS"].dropna().tolist(), 2)

  # EPS季增率
  quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)

  # 轉換日期
  dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]

  data = {
      '季日期': dates[:len(quarterly_revenue_growth)],
      '營收成長率': quarterly_revenue_growth.tolist(),
      'EPS': quarterly_eps[0:3].tolist(),
      'EPS 季增率': quarterly_eps_growth[0:3].tolist()
  }

  return data

print(stock_fundamental("3526.TWO"))

{'季日期': ['2023-09-30', '2023-06-30', '2023-03-31'], '營收成長率': [0.14, 0.39, -0.18], 'EPS': [3.27, 3.07, 1.21], 'EPS 季增率': [0.07, 1.54, -0.28]}


### 5️⃣ 取得新聞資料

In [ ]:
# 新聞資料
def stock_news(stock_name ="大盤"):
  if stock_name == "大盤":
    stock_name="台股 -盤中速報"

  data=[]
  # 取得 Json 格式資料
  json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_name}&limit=5&page=1').json()

  # 依照格式擷取資料
  items=json_data['data']['items']
  for item in items:
      # 網址、標題和日期
      news_id = item["newsId"]
      title = item["title"]
      publish_at = item["publishAt"]
      # 使用 UTC 時間格式
      utc_time = dt.datetime.utcfromtimestamp(publish_at)
      formatted_date = utc_time.strftime('%Y-%m-%d')
      # 前往網址擷取內容
      url = requests.get(f'https://news.cnyes.com/'
                        f'news/id/{news_id}').content
      soup = BeautifulSoup(url, 'html.parser')
      p_elements=soup .find_all('p')
      # 提取段落内容
      p=''
      for paragraph in p_elements[4:]:
          p+=paragraph.get_text()
      data.append([stock_name, formatted_date ,title,p])
  return data

print(stock_news("台積電"))

[['台積電', '2024-03-04', '〈台幣〉股匯雙漲 往31.5元關卡靠攏 攀1周高點', '觀察主要貨幣今天對美元的表現，新台幣、泰銖早盤均升值逾 0.1%，韓元、人民幣、新加坡幣都在平盤附近整理，日元則貶值 0.03%。本週國際重點財經大事包含美國總統初選「超級星期二」，以及 2 月非農報告及失業率、ISM 非製造業指數、上週失業金等經濟數據。此外，聯準會 (Fed) 褐皮書定於本週出爐，市場同時關注 Fed 主席鮑爾的國會證詞，以及多位 Fed 官員的發言。歐洲央行 (ECB) 和加拿大央行也將在本週公布最新利率決議。台股方面，在台積電 ADR 大漲 4% 創波段新高的激勵下，台積電 (2330-TW) 今天一度衝上 715 元寫新天價，大盤同步飛越 19000 點大關，漲點更放大至 300 點以上，最高來到 19272 點，改寫歷史新高。本周國內有多場法說會，可望吸引市場買盤押寶，加上第一季作帳行情開跑，為多頭再添火力。下一篇'], ['台積電', '2024-03-04', '戴爾蝴碟效應漲勢擴散 繼技嘉、廣運後最便宜的AI股換「它」接棒！', 'AI 股將輪動輪漲 基期愈低接棒大漲機會愈大！今日台積電大漲再創新高價，2/15 的 709 元已非天價。無疑是宣告，AI 股輪動輪漲的趨勢仍在運行之中。看看我們今年一路提示，通吃 AI PC、AI 手機、AI 伺服器的散熱族群—奇鋐 (3017-TW)、雙鴻(3324-TW)、健策(3653-TW)、技嘉(2376-TW)，皆已如預期已是遍地開花。再看近期強力鎖定的另一檔散熱股廣運(6125-TW) 更是有後來居上之勢！這是為何？關鍵就在股價的基期位置。所以，3 月主升股將換誰接棒？該如何選？誰將複製 DELL 大漲模式，接棒廣運成為 3 月最猛的主升股？目前江江已鎖定具「低基期 + 大爆發力成長」的新散熱潛龍股已經蓄勢待發，若還不知道該如何應對盤勢變化、不知逢低卡位那隻 AI 潛龍股？手上持股不知如何操作。那建議你務必要鎖定江江在 Line @，將有更進一步的訊息給大家了解。想了解還未起漲的市場主流，同步了解大盤多空轉折點及學習預測技術分析，都可加入密切鎖定 Line@粉絲團關注。接棒廣運重返榮耀！ 散熱裡超大黑馬—高力 (8996-TW)高力為國內唯一掛牌之生產板式熱交換器廠商，為全球前五大的熱

### 6️⃣ 爬取股號、股名對照表

In [ ]:
# 取得全部股票的股號、股名
def stock_name(type=2):
  print("線上讀取股號、股名、及市場別")

  response = requests.get(f'https://isin.twse.com.tw/isin/C_public.jsp?strMode={type}')
  url_data = BeautifulSoup(response.text, 'html.parser')
  stock_company = url_data.find_all('tr')

  # 資料處理
  data = [
      (row.find_all('td')[0].text.split('\u3000')[0].strip(),
        row.find_all('td')[0].text.split('\u3000')[1],
        row.find_all('td')[3].text.strip())
      for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
  ]

  df = pd.DataFrame(data, columns=['股號', '股名', '市場別'])

  return df

name_df = stock_name()
name_df = pd.concat([name_df, stock_name(type=4)])

線上讀取股號、股名、及市場別
線上讀取股號、股名、及市場別


### 7️⃣ 取得股票名稱

In [ ]:
# 取得股票名稱
def get_stock_name(stock_id, name_df):
    stock_name = name_df.set_index('股號').loc[stock_id, '股名']
    stock_type = name_df.set_index('股號').loc[stock_id, '市場別']
    return stock_name, stock_type


print(name_df.head())
print("--------------------------")
print(get_stock_name("3526",name_df))

     股號  股名 市場別
0  1101  台泥  上市
1  1102  亞泥  上市
2  1103  嘉泥  上市
3  1104  環泥  上市
4  1108  幸福  上市
--------------------------
('凡甲', '上櫃')


### 8️⃣ 建構 GPT 3.5 模型

In [ ]:
# 建立 GPT 3.5-16k 模型
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.type} 錯誤\n{err.message}"
    return reply

# 建立訊息指令(Prompt)
def generate_content_msg(stock_id, name_df):
    # 獲取股票名稱和類型
    stock_name, stock_type = get_stock_name(stock_id, name_df) if stock_id != "大盤" else (stock_id, None)

    # 根據股票類型調整股票ID
    if stock_type == "上市":
        stock_id += ".TW"
    elif stock_type == "上櫃":
        stock_id += ".TWO"

    # 獲取股票價格和新聞資料
    price_data = stock_price(stock_id)
    news_data = stock_news(stock_name)

    # 生成內容訊息
    content_msg = f'請依據以下資料來進行分析並給出一份完整的分析報告:\n'
    content_msg += f'近期價格資訊:\n {price_data}\n'

    if stock_id != "大盤":
        stock_value_data = stock_fundamental(stock_id)
        content_msg += f'每季營收資訊：\n {stock_value_data}\n'

    content_msg += f'近期新聞資訊: \n {news_data}\n'
    content_msg += f'請給我{stock_name}近期的趨勢報告,請以詳細、嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'

    return content_msg


# StockGPT
def stock_gpt(stock_id, name_df=name_df):
    content_msg = generate_content_msg(stock_id, name_df)

    msg = [{
        "role": "system",
        "content": f"你現在是一位專業的證券分析師, 你會統整近期的股價\
      、基本面、新聞資訊等方面並進行分析, 然後生成一份專業的趨勢分析報告"
    }, {
        "role": "user",
        "content": content_msg
    }]

    reply_data = get_reply(msg)

    return reply_data


### 9️⃣ 大盤趨勢報告

In [ ]:
reply = stock_gpt(stock_id="大盤")
print(reply)

[*********************100%%**********************]  1 of 1 completed


根據最近的價格資訊，我們可以看到大盤收盤價在短時間內有波動，但整體呈現上升趨勢。截至 2024 年 3 月 4 日，大盤收盤價位於 19278.83984375 點，相較於前幾個交易日有明顯上升。每日報酬率方面，我們可以觀察到對應的漲跌價差，2024 年 2 月 26 日至 3 月 4 日間的漲跌價差分別為 58.861328125、-93.640625、112.359375、-30.83984375、342.91015625 點。從漲跌價差的變化來看，市場波動明顯。

就新聞資訊而言，有關全球 CIS 龍頭日商索尼在台積電日本熊本新廠下單的消息顯示，半導體產業持續活躍，並伴隨著在地化生產趨勢，這對大盤有利，對於半導體產業的展望應予以關注。另外，AI 應用帶動的相關製造需求也帶來正面的市場因素。然而，投信資金有趁著電子股衝高後獲利了結的現象，顯示投資者在市場波動時持續保持警覺。

從技術面分析來看，股價淨值比為 2.26 倍，且預估台股指數可達 20121 點，上漲空間僅剩 6.2%。考量到短期內大盤的波動，投資者應謹慎評估市場狀況及調整持股策略。

綜合分析，雖然大盤短期內有波動，但整體呈現上升趨勢。在半導體產業持續活躍、AI 應用帶動的製造需求增加的情況下，對於市場帶來正面影響。然而，投信資金的獲利了結以及技術面上的警示，提醒投資者需謹慎評估風險並適時調整投資策略。

以上報告僅供參考，投資者仍應自行進行詳細分析並謹慎評估風險後進行投資決策。


### 🔟 上市上櫃都 OK

In [ ]:
reply = stock_gpt(stock_id="3526")
print(reply)

[*********************100%%**********************]  1 of 1 completed


【凡甲近期趨勢分析報告】

一、價格趨勢分析
根據最近一段時間的股價表現來看，凡甲股票的收盤價在2024年2月23日至3月4日之間波動不大，最高價為212.5元，最低價為208.0元，整體漲跌幅度較小。每日報酬率方面，2月26日至3月1日期間報酬率波動也較小，整體呈現穩定的走勢。

二、基本面分析
從凡甲近三季度的營收表現來看，公司營收呈現穩步增長的趨勢，2023年3月31日之負成長率變為2023年9月30日的正成長率，顯示了公司整體業務發展良好的跡象。此外，每股盈餘（EPS）亦有穩步增長，顯示公司盈利能力的提升。

三、新聞事件分析
根據最近新聞資訊，凡甲公司涉及的公告主要集中在公司內部決策及董事會重要決議等方面，並有轉換公司債相關公告。這些內部管理相關的消息對公司股價走勢影響較小。

四、總結與建議
綜合以上分析，整體來看，凡甲的股價走勢較為平穩，基本面表現良好，且公司內部管理穩健。建議投資者應密切關注公司的業務拓展以及內部管理的進一步優化，以獲取更穩定的投資收益。

以上報告僅供參考，投資者在做出投資決策前還應對更多資訊進行研究及分析。
